In [ ]:
import torch
print(torch.__version__)
print(torch.autograd)




In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from tqdm.keras import TqdmCallback

In [ ]:

# Verify GPU availability
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Configure data generators
from tensorflow.keras.applications.efficientnet import preprocess_input

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:

# Dataset paths
base_dir = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
train_dir = f'{base_dir}/train'
valid_dir = f'{base_dir}/valid'
test_dir = f'{base_dir}/test'
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

In [ ]:
# The number of samples
num_samples = train_generator.samples

# Number of classes
num_classes = train_generator.num_classes

# Batch size
batch_size = train_generator.batch_size

# Number of batches per epoch
steps_per_epoch = train_generator.samples // train_generator.batch_size

print(f"Training samples: {num_samples}")
print(f"Number of classes: {num_classes}")
print(f"Steps per epoch: {steps_per_epoch}")

In [ ]:



from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.regularizers import l2

with tf.device('/GPU:0'):
    # Load base model
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Freeze first 50% of layers initially
    for layer in base_model.layers[:int(len(base_model.layers)*0.5)]:
        layer.trainable = False
    
    # Enhanced custom head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Feature processing block
    x = Dense(512, kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dropout(0.6)(x)
    
    # Attention to subtle artifacts
    x = Dense(256, kernel_regularizer=l2(0.005))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dropout(0.5)(x)
    
    # Final classification
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Custom optimizer configuration
    optimizer = Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999)
    
    model.compile(optimizer = Adam(learning_rate=1e-4),
              loss = 'binary_crossentropy',
              metrics = ['acc'])



In [ ]:
model.summary()

In [ ]:
import warnings
from keras.callbacks import EarlyStopping

# Suppress warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # Train with GPU and show progress bar
    history = model.fit(
        train_generator,
        epochs=2,
        validation_data=valid_generator,
        verbose=1  # this enables the progress bar
    )


In [ ]:
model.save('this_is_final.h5')

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink('this_is_final.h5')

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')


In [ ]:



# Generate predictions and evaluation reports
test_generator.reset()
y_pred = model.predict(test_generator, verbose=1)
y_pred = np.round(y_pred).flatten()
y_true = test_generator.classes

print('\nConfusion Matrix:')
print(confusion_matrix(y_true, y_pred))

print('\nClassification Report:')
print(classification_report(y_true, y_pred))

# Plot training history
plt.figure(figsize=(12, 4))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Dataset paths
#just checking another dataset accuracy
base_dir2 = '/kaggle/input/facess/Human Faces Dataset'
train_dir2 = f'{base_dir2}/train'
train_generator2 = train_datagen.flow_from_directory(
    train_dir2,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
)


In [ ]:
test_loss, test_acc = model.evaluate(train_generator2, verbose=1)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')


In [ ]:
model.summary()